# ipywidgets learning

Goal:  build an interface for programming Evernote using IPython widgets.

Let me list the relevant tutorial materials:

* [ipywidgets/Index.ipynb at master · ipython/ipywidgets](https://github.com/ipython/ipywidgets/blob/master/examples/notebooks/Index.ipynb)

In [ ]:
from __future__ import print_function

from ipywidgets import (interact, interactive, fixed)
import ipywidgets as widgets

from IPython.display import (HTML, display, Javascript)

# for function annotation in Python 3
from IPython.utils.py3compat import annotate

What is the best way to get a list of widgets?  According to [ipywidgets/Widget List.ipynb at master · ipython/ipywidgets](https://github.com/ipython/ipywidgets/blob/master/examples/notebooks/Widget%20List.ipynb):

```Python
import ipywidgets as widgets
widgets.Widget.widget_types.values()
```



In [ ]:
widgets.Widget.widget_types.values()

In [ ]:
widgets.Widget.widget_types.values()[0].__name__

In [ ]:
import ipywidgets as widgets
[w.__name__ for w in widgets.Widget.widget_types.values()]

In [ ]:
standard_handlers = set(['on_displayed', 'on_msg', 'on_trait_change', 'on_widget_constructed'])

# It seems one will use on_trait_change a lot. I'll need to learn more the other three standard ones
# extra handlers: on_msg, on_displayed, on_widget_constructed


# Text -> on_submit
# Button -> on_click

for w in widgets.Widget.widget_types.values():
    print ((w,set([m for m in dir(w) if m.startswith("on_")]) - standard_handlers))

In [ ]:
dir(widgets)

## interact

In [ ]:
import math
i = interact(lambda x: math.factorial(x),
         x=widgets.IntSlider(min=0,max=2000,step=1,value=10));


In [ ]:
type(i.widget)

In [ ]:
# I can mess with the value of the input -- and the IntSlider moves, the output doesn't change above
i.widget.children[0].value =2

I thought with `interact`, I could change `x` and the widget would change....nope.

## interactive

In [ ]:
def f(a,b):
    return a+b

In [ ]:
w = interactive(f, a=10, b=20)

In [ ]:
display(w)

In [ ]:
# how can I control the parameters in Python?

In [ ]:
c0 = w.children[0]
(c0.description, c0.value)

In [ ]:
c0.value = 23

In [ ]:
w.kwargs

In [ ]:
w.result

# using widgets without JS

[Question about how to use existing widgets without necessarily creating custom widgets · Issue #263 · ipython/ipywidgets](https://github.com/ipython/ipywidgets/issues/263)

In [ ]:
%matplotlib notebook
from ipywidgets import HBox, VBox, Button
from IPython.display import display
from matplotlib import pyplot as plt

red = Button(description='Make Red')
green = Button(description='Make Green')

fig, ax = plt.subplots()
ax.plot([1,2,3])

def make_red(button):
    ax.lines[0].set_color('red')
    ax.figure.canvas.draw()

def make_green(button):
    ax.lines[0].set_color('green')
    ax.figure.canvas.draw()

red.on_click(make_red)
green.on_click(make_green)
display(HBox([red, green]))

In [ ]:
int_range = widgets.IntSlider()
display(int_range)

def on_value_change(name, value):
    print(value)

int_range.on_trait_change(on_value_change, 'value')

In [ ]:
from __future__ import print_function

t_widget = widgets.Text()
display(t_widget)

num_events = 0

def on_text_change(name, value):
    
    global num_events 
    print ('\r{}: {}'.format(num_events, value), end="")
    num_events +=1
    
t_widget.on_trait_change(on_text_change, 'value')

In [ ]:
# list of traits -- lot more traits than I expected.
t_widget.trait_names()

In [ ]:
t_widget.background_color = '#FFFF00'
t_widget.border_color = 'blue'

In [ ]:
t_widget.value = "dddd"

In [ ]:
from collections import OrderedDict
import datetime

def inv_dict(d):
    return dict([(v,k) for (k,v) in d.items()])

w = widgets.Dropdown(
    options=OrderedDict([('Apple', 1), ('Orange', 2), ('Plum', 3), ('Watermelon',4)]),
    value=2,
    description='Number:',
)

def w_changed(name, value):
    d = inv_dict(w.options)
    
    t_widget.value = unicode(unicode(value) + ":" + d.get(value,''))

w.on_trait_change(w_changed, 'value')

t_widget = widgets.Text()

display(w, t_widget)



In [ ]:
OrderedDict([(v,k) for (k,v) in w.options.items()])

In [ ]:
w.value = 4

In [ ]:
w.options = OrderedDict(w.options.items() + [('Peaches', 5)])

In [ ]:
from IPython.display import HTML
HTML("<div>hello</div>")

In [ ]:
qgrid.show_grid(spy)

# how to use/extend qqgrid widget?

How to use `QGridWidget`, identified in [Add an IPython widget by blink1073 · Pull Request #27 · quantopian/qgrid](https://github.com/quantopian/qgrid/pull/27/files#diff-2559840b81815e696a22fbaee6294b66R172)?

My impression is that we still use the main `qgrid.show_grid` but that we can make use of events in the underlying `QGridWidget`.  Let's see. 

# little detour into learning about making custom widgets

In [ ]:
from ipywidgets import widgets
import traitlets

In [ ]:
class HelloView(widgets.DOMWidget):
    _view_name = traitlets.Unicode("HelloView", sync=True)
    value = traitlets.Unicode("Hello World", sync=True)


In [ ]:
%%javascript
requirejs(['nbextensions/widgets/widgets/js/widget',
         'nbextensions/widgets/widgets/js/manager',], function(widget, manager){
    
    var HelloView = widget.DOMWidgetView.extend({
        
        render: function(){
            this.value_changed();
            this.model.on('change:value', this.value_changed, this);
        },
        
        value_changed: function () {
            this.$el.text(
              this.model.get('value')
            )
            
        }
        
    });
    
    manager.WidgetManager.register_widget_view('HelloView', HelloView);
    
})

In [ ]:
h = HelloView()
h

In [ ]:
h.value = "dog"

In [ ]:
# jquery-ui spinner

from nbfiddle import Fiddle

In [ ]:
# datepicker

Fiddle(
  html = """
    <input type="text" name="date" class="date1"/>
  """,
    
  js = """
    $('#{{div_id}} input.date1').datepicker();
  """,
  jslibs = (("jquery-ui", "https://code.jquery.com/ui/1.11.4/jquery-ui.min", "jqu"),),
  csslibs = ('https://cdnjs.cloudflare.com/ajax/libs/jqueryui/1.11.4/jquery-ui.css',)
)

In [ ]:
# datepicker w/o adding jquery-ui -- built in?

Fiddle(
  html = """
    <input type="text" name="date" class="date1"/>
  """,
    
  js = """
    $('#{{div_id}} input.date1').datepicker();
  """,
  csslibs = ('https://cdnjs.cloudflare.com/ajax/libs/jqueryui/1.11.4/jquery-ui.css',)
)

In [ ]:
# spinner
# https://jqueryui.com/spinner/


Fiddle(
  html = """
<p>
  <label for="spinner">Select a value:</label>
  <input class="spinner" name="value">
</p>
<p>
  <label>Value:</label>
  <span class="reflected_val"></span>
</p>
  """,
    
  js = """
    var spinner = $( "#{{div_id}} .spinner" ).spinner();
    spinner.spinner( "value", 2 );

    var reflected_val = $("#{{div_id}} .reflected_val");
   
    spinner.on( "spinchange", function( event, ui ) {
        console.log(ui);
        reflected_val.text(spinner.val());
    } );
    
    spinner.on("spin", function(event, ui){
        reflected_val.text(ui.value);
    
    })
  """,
  jslibs = (("jquery-ui", "https://code.jquery.com/ui/1.11.4/jquery-ui.min", "jqu"),),
  csslibs = ('https://cdnjs.cloudflare.com/ajax/libs/jqueryui/1.11.4/jquery-ui.css',)
)

# location of files installed by ipywidgets, etc.


I'm trying to understand how various **things** are packaged:

* nbextensions
* IPython widgets

Recall 

* Python env:
* IPython env

What's the meaning of `nbextensions/widgets/widgets/js/widget`?

In [1]:
%%bash

ls -al ~/.ipython ~/.jupyter

/Users/raymondyee/.ipython:
total 24
drwxr-xr-x    7 raymondyee  staff   238 May 10  2015 .
drwxr-xr-x+ 137 raymondyee  staff  4658 Jan 28 21:50 ..
-rw-r--r--@   1 raymondyee  staff  6148 May 10  2015 .DS_Store
-rw-r--r--    1 raymondyee  staff   195 Aug 13  2014 README
drwxr-xr-x    2 raymondyee  staff    68 Aug 13  2014 extensions
drwxr-xr-x    2 raymondyee  staff    68 Aug 13  2014 nbextensions
drwxr-xr-x    9 raymondyee  staff   306 Jan 29 08:45 profile_default

/Users/raymondyee/.jupyter:
total 40
drwxr-xr-x    8 raymondyee  staff   272 Dec  6 15:03 .
drwxr-xr-x+ 137 raymondyee  staff  4658 Jan 28 21:50 ..
-rw-r--r--    1 raymondyee  staff   333 Dec  6 14:55 jupyter_nbconvert_config.json
-rw-r--r--    1 raymondyee  staff   327 Dec  6 14:55 jupyter_nbconvert_config.py
-rw-r--r--    1 raymondyee  staff   411 Dec  6 14:55 jupyter_notebook_config.json
-rw-r--r--    1 raymondyee  staff   331 Dec  6 14:55 jupyter_notebook_config.py
-rw-r--r--    1 raymondyee  staff    26 Aug 14 11:20 mi

In [2]:
from jupyter_core.paths import jupyter_config_dir, jupyter_data_dir
jupyter_config_dir(), jupyter_data_dir()

('/Users/raymondyee/.jupyter', '/Users/raymondyee/Library/Jupyter')

In [3]:
%%bash

ls /Users/raymondyee/Library/Jupyter/nbextensions/

codemirrormode
config
publishing
qgridjs
slidemode
styling
testing
usability


In [ ]:
import notebook.nbextensions

In [ ]:
dir(notebook.nbextensions)

# Cleaning up widgets

In [ ]:
assert False

In [ ]:
%%javascript

// close widgets by clickling on clse buttons.
$('div.widget-area button.close:visible').click();